In [19]:
import matplotlib.pyplot as plt
import seaborn as sns
import polars as pl
import numpy as np
from numpy.typing import NDArray
from typing import Sequence, Any, Tuple, Union, List, Dict, Optional, TypeVar, Callable, Iterable, cast
from datetime import datetime, timedelta

In [2]:
from stonesoup.models.transition.linear import CombinedLinearGaussianTransitionModel, \
                                               ConstantVelocity
from stonesoup.types.groundtruth import GroundTruthPath, GroundTruthState
from stonesoup.types.detection import TrueDetection
from stonesoup.types.detection import Clutter
from stonesoup.models.measurement.linear import LinearGaussian
from stonesoup.types.detection import Detection
from stonesoup.plotter import AnimatedPlotterly
from stonesoup.predictor.kalman import KalmanPredictor
from stonesoup.updater.kalman import KalmanUpdater
from stonesoup.hypothesiser.distance import DistanceHypothesiser
from stonesoup.measures import Mahalanobis
from stonesoup.dataassociator.neighbour import GlobalNearestNeighbour
from stonesoup.deleter.error import CovarianceBasedDeleter
from stonesoup.types.state import GaussianState
from stonesoup.initiator.simple import MultiMeasurementInitiator

In [5]:
import awkward as ak

data = ak.from_parquet("detections.parquet")
display(data.typestr)

'795 * var * {x: int64, y: int64, w: int64, h: int64, area: float64, cX: int64, cY: int64}'

In [ ]:
# kalman
transition_model = CombinedLinearGaussianTransitionModel(
    [ConstantVelocity(0.05), ConstantVelocity(0.05)])
measurement_model = LinearGaussian(ndim_state=4,
                                   mapping=(0, 2),
                                   noise_covar=np.diag([0.75, 0.75]))
predictor = KalmanPredictor(transition_model)
updater = KalmanUpdater(measurement_model)

hypothesiser = DistanceHypothesiser(predictor,
                                    updater,
                                    measure=Mahalanobis(),
                                    missed_distance=3)
data_associator = GlobalNearestNeighbour(hypothesiser)

deleter = CovarianceBasedDeleter(covar_trace_thresh=4)
initiator = MultiMeasurementInitiator(
    prior_state=GaussianState([[0], [0], [0], [0]], np.diag([0, 1, 0, 1])),
    measurement_model=measurement_model,
    deleter=deleter,
    data_associator=data_associator,
    updater=updater,
    min_points=2,
)

In [27]:
detections_coords = data[:, :, ["cX", "cY"]]
# use unzip to separate the x and y coordinates
# display(detections_coords)

In [30]:
# https://github.com/rlabbe/filterpy
import jax.numpy as jnp
from jax import random, vmap, jit, grad, value_and_grad
import numpy as np
import jax
import chex
from jaxtyping import Array, Shaped, Num, Int, Float, Bool

In [32]:
@chex.dataclass
class Initiator:
    # tentative tracks are temporary tracks maintained by the initiator that
    # have been initialized but not yet confirmed
    tentative_tracks: Num[Array, "... 2"]


In [49]:
import filterpy
from filterpy.kalman import KalmanFilter

# input [x y]
# state [x y dx/dt dy/dt]


# yapf: disable
def F_cv(dt: float|int):
    return np.array([[1, 0, dt, 0],
                     [0, 1, 0, dt],
                     [0, 0, 1, 0],
                     [0, 0, 0, 1]])
# yapf: enable


def H_cv():
    return np.array([[1, 0, 0, 0], [0, 1, 0, 0]])


kf = KalmanFilter(4, 2)
T = 1.0
kf.F = F_cv(T)
kf.H = H_cv()
kf.R = np.diag([0.75, 0.75])
kf.Q = np.diag([0.05, 0.05, 0.05, 0.05])
# a simple constant velocity model
# let's have a hypothesis of the initial velocity
# is 0.05 unit/dt in both x and y directions
kf.x = np.array([0, 0, 0.05, 0.05])
kf.predict()
# x now becomes x prior

# kf.update([0.15, 0.15])
# x_posterior 
# when x is updated, it becomes x_posterior

array([[0.],
       [0.],
       [0.],
       [0.]])

In [48]:
# from predict state to measurement
kf.x_prior # predicted state
# predicted measurement
# https://peps.python.org/pep-0465/
predicted_measurement = kf.H @ kf.x
display(predicted_measurement)
# compare the predicted measurement with the actual measurement
# with mahalanobis distance
actual_measurement = np.array([0.12, 0.12])
# or just euclidean distance
mahalanobis_distance = np.sqrt((actual_measurement - predicted_measurement) @ np.linalg.inv(kf.R) @ (actual_measurement - predicted_measurement))
display(mahalanobis_distance)
# use mahalanobis distance as a loss function to determine the best match
# Hungarian algorithm
from scipy.optimize import linear_sum_assignment
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linear_sum_assignment.html
# when we get a successively detected object, we can move it into the confirmed tracks
# and remove it from the tentative tracks
# Well, it's more like two GNNs, one for the tentative tracks and one for the confirmed tracks
# cascaded GNN, interesting...

array([0.1, 0.1])

0.032659863237109024